# Imports

In [150]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Analisis preliminar y Limpieza

In [151]:
data = pd.read_csv(Path(os.getcwd()).joinpath('dataset_SCL.csv'))
print(f'Shape of dataframe: {data.shape}')
data.sample(frac=0.0001, random_state=12)

Shape of dataframe: (68206, 18)


C:\Users\Bastian\AppData\Local\Temp\ipykernel_8816\4224886279.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(Path(os.getcwd()).joinpath('dataset_SCL.csv'))


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
58560,2017-11-29 08:35:00,148,SCEL,SCCF,LAN,2017-11-29 08:29:00,148,SCEL,SCCF,LAN,29,11,2017,Miercoles,N,Grupo LATAM,Santiago,Calama
63027,2017-12-04 00:35:00,9643,SCEL,SBGR,TAM,2017-12-04 00:45:00,9643,SCEL,SBGR,TAM,4,12,2017,Lunes,I,Grupo LATAM,Santiago,Sao Paulo
66340,2017-12-16 15:35:00,410,SCEL,SUMU,LAN,2017-12-16 15:57:00,410.0,SCEL,SUMU,LAN,16,12,2017,Sabado,I,Grupo LATAM,Santiago,Montevideo
63215,2017-12-01 19:15:00,25,SCEL,SCTE,SKU,2017-12-01 20:00:00,25,SCEL,SCTE,SKU,1,12,2017,Viernes,N,Sky Airline,Santiago,Puerto Montt
49004,2017-09-05 19:10:00,704,SCEL,LEMD,LAN,2017-09-05 19:08:00,704,SCEL,LEMD,LAN,5,9,2017,Martes,I,Grupo LATAM,Santiago,Madrid
67741,2017-12-24 16:30:00,54,SCEL,SCFA,LAW,2017-12-24 16:32:00,54.0,SCEL,SCFA,JMR,24,12,2017,Domingo,N,Latin American Wings,Santiago,Antofagasta
6446,2017-02-16 16:16:00,116,SCEL,SKBO,AVA,2017-02-16 16:20:00,116,SCEL,SKBO,AVA,16,2,2017,Jueves,I,Avianca,Santiago,Bogota


Vemos que, a priori, en general la informacion de programacion es igual a la de operacion de vuelo (salvo la hora de vuelo, que es el problema a predecir), pero hay casos donde la aerolinea no es la misma. Addemas, 'OPERA' pareciera indicar la misma informacion que 'Emp-I' y no 'Emp-O', ya que Latin American Wings probablemente corresponde con la sigla LAW y no con JMR.

In [152]:
# get data where Des-I is different from Des-O
data[data["Des-I"] != data["Des-O"]].sample(frac = 0.3, random_state = 42)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
20444,2017-04-14 22:00:00,289,SCEL,SCTE,LAN,2017-04-14 22:01:00,289,SCEL,SCCI,LXP,14,4,2017,Viernes,N,Grupo LATAM,Santiago,Punta Arenas
67600,2017-12-02 11:15:00,7811,SCEL,SAEZ,LAN,2017-12-02 11:21:00,7811.0,SCEL,SABE,DSM,2,12,2017,Sabado,I,Grupo LATAM,Santiago,Buenos Aires
18293,2017-04-14 18:40:00,25,SCEL,SCTE,SKU,2017-04-14 18:48:00,25,SCEL,SCQP,SKU,14,4,2017,Viernes,N,Sky Airline,Santiago,Temuco
55144,2017-10-06 00:55:00,1260,SCEL,SEQU,LAN,2017-10-06 02:15:00,1260,SCEL,SEQM,LAN,6,10,2017,Viernes,I,Grupo LATAM,Santiago,Quito
1397,2017-01-17 19:20:00,25,SCEL,SCTE,SKU,2017-01-17 19:46:00,25,SCEL,SCQP,SKU,17,1,2017,Martes,N,Sky Airline,Santiago,Temuco
25422,2017-05-12 08:25:00,271,SCEL,SCBA,LAN,2017-05-12 08:36:00,271,SCEL,SCTE,LXP,12,5,2017,Viernes,N,Grupo LATAM,Santiago,Puerto Montt
38135,2017-07-17 16:00:00,7813,SCEL,SABE,LAN,2017-07-17 16:41:00,1113,SCEL,SAEZ,LAN,17,7,2017,Lunes,I,Grupo LATAM,Santiago,Buenos Aires
61463,2017-11-06 06:00:00,1031,SCEL,SCIE,LAW,2017-11-06 08:01:00,1031,SCEL,SCTE,JMR,6,11,2017,Lunes,N,Latin American Wings,Santiago,Puerto Montt


Aqui vemos que en algunos casos el destino de vuelo tambien cambia entre la informacion de programacion y operacion. Ademas, en este caso 'SIGLADES' no indica la misma informacion de 'Des-I', si no que la de 'Des-O', lo que se puede ver porque las siglas de Puerto Montt y Temuco coinciden para 'Des-O' y no para 'Des-I'.

In [153]:
# get data where Ori-I is different from Ori-O
data[data["Ori-I"] != data["Ori-O"]]

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES


Para la ciudad de origen, los datos de programacion vs operacion coinciden perfectamente

In [154]:
# get data where Vlo-I is different from Vlo-O
data[data["Vlo-I"] != data["Vlo-O"]].sample(frac = 0.002, random_state = 42)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
63920,2017-12-25 04:00:00,170,SCEL,SCDA,SKU,2017-12-25 03:50:00,170.0,SCEL,SCDA,SKU,25,12,2017,Lunes,N,Sky Airline,Santiago,Iquique
65253,2017-12-07 17:32:00,213,SCEL,SCIE,LAN,2017-12-07 17:41:00,213.0,SCEL,SCIE,LXP,7,12,2017,Jueves,N,Grupo LATAM,Santiago,Concepcion
64173,2017-12-21 21:10:00,503,SCEL,SAEZ,SKU,2017-12-21 22:03:00,503.0,SCEL,SAEZ,SKU,21,12,2017,Jueves,I,Sky Airline,Santiago,Buenos Aires
64486,2017-12-24 09:38:00,61,SCEL,SCTE,LAN,2017-12-24 09:45:00,61.0,SCEL,SCTE,LAN,24,12,2017,Domingo,N,Grupo LATAM,Santiago,Puerto Montt


En el caso del numero de vuelo, ocurre que algunos datos fueron guardados como float y cargados como texto, por lo que habria que cambiarlos a int, pero como pandas no los transformó a numerico automaticamente, probablemente haya otro problema tambien

In [155]:
# get data where Vlo-O cannot be converted into numeric
not_num_mask = pd.to_numeric(data["Vlo-O"], errors='coerce').isna() 
data.loc[not_num_mask, :].sample(frac = 0.5, random_state = 42)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,NaN,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima
13906,2017-03-30 10:30:00,71,SCEL,SCIE,SKU,2017-03-30 10:50:00,71R,SCEL,SCIE,SKU,30,3,2017,Jueves,N,Sky Airline,Santiago,Concepcion
22167,2017-04-06 21:10:00,43,SCEL,SCIE,LAW,2017-04-06 21:24:00,43R,SCEL,SCIE,JMR,6,4,2017,Jueves,N,Latin American Wings,Santiago,Concepcion
55354,2017-10-19 03:30:00,502,SCEL,KMIA,LAN,2017-10-19 03:33:00,502R,SCEL,KMIA,LAN,19,10,2017,Jueves,I,Grupo LATAM,Santiago,Miami
32196,2017-06-29 12:40:00,492,SCEL,SACO,LAN,2017-06-29 12:27:00,492R,SCEL,SACO,LAN,29,6,2017,Jueves,I,Grupo LATAM,Santiago,Cordoba
44939,2017-09-28 21:30:00,846,SCEL,KIAH,UAL,2017-09-28 21:44:00,846A,SCEL,KIAH,UAL,28,9,2017,Jueves,I,United Airlines,Santiago,Houston
24304,2017-05-11 10:00:00,802,SCEL,SPJC,SKU,2017-05-11 10:36:00,802R,SCEL,SPJC,SKU,11,5,2017,Jueves,I,Sky Airline,Santiago,Lima
39225,2017-08-07 19:00:00,591P,SCEL,LFPG,PUE,2017-08-07 20:35:00,591P,SCEL,LFPG,PUE,7,8,2017,Lunes,I,Plus Ultra Lineas Aereas,Santiago,Paris


Vemos que los valores que no pueden ser convertidos se deben a que tienen letras luego del numero, esto tambien ocurre en 'Vlo-I'; ademas, se nota un NaN.

Entonces, nos deshacemos de las letras en los numeros de vuelo

In [156]:
# only keep numeric values in the string
data["Vlo-O"] = data["Vlo-O"].map(lambda x: ''.join([i for i in str(x) if i.isdigit()]))
data["Vlo-I"] = data["Vlo-I"].map(lambda x: ''.join([i for i in str(x) if i.isdigit()]))

# get data where Vlo-O cannot be converted into numeric
not_num_mask = pd.to_numeric(data["Vlo-O"], errors='coerce').isna() 
data.loc[not_num_mask, :]

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima


Vemos que ahora solo hay un valor no numerico, que corresponde al NaN de antes; el cual se eliminará

In [157]:
data.drop(labels=6068, axis=0, inplace=True)

Ahora vemos que numeros de vuelo son distintos

In [158]:
# get data where Vlo-I is different from Vlo-O
data[data["Vlo-I"] != data["Vlo-O"]].sample(frac = 0.001, random_state = 42)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
65339,2017-12-18 07:25:00,231,SCEL,SCQP,LAN,2017-12-18 07:38:00,2310,SCEL,SCQP,LXP,18,12,2017,Lunes,N,Grupo LATAM,Santiago,Temuco
67392,2017-12-09 12:20:00,1193,SCEL,SCCI,LAN,2017-12-09 12:20:00,11930,SCEL,SCCI,LXP,9,12,2017,Sabado,N,Grupo LATAM,Santiago,Punta Arenas
67949,2017-12-12 05:00:00,26,SCEL,SCFA,JAT,2017-12-12 05:16:00,260,SCEL,SCFA,JAT,12,12,2017,Martes,N,JetSmart SPA,Santiago,Antofagasta
67710,2017-12-07 05:00:00,50,SCEL,SCFA,LAW,2017-12-07 04:55:00,500,SCEL,SCFA,JMR,7,12,2017,Jueves,N,Latin American Wings,Santiago,Antofagasta
64173,2017-12-21 21:10:00,503,SCEL,SAEZ,SKU,2017-12-21 22:03:00,5030,SCEL,SAEZ,SKU,21,12,2017,Jueves,I,Sky Airline,Santiago,Buenos Aires


Vemos que ahora hay un problema con numeros de vuelo con un 0 extra al final, si se eliminace de la misma manera que el anterior, se corre el riesgo de eliminar un 0 que no sea extra, haciendo que un vuelo que si era igual, ahora no lo sea; por lo que se trabajará solo con los que ya son distintos

In [159]:
# create temp dataframe where Vlo-I is different from Vlo-O
data_temp = data[data["Vlo-I"] != data["Vlo-O"]]

# ignore the last character if it is a '0'
data_temp["Vlo-O"] = data_temp["Vlo-O"].map(lambda x: x[:-1] if x[-1] == '0' else x)

# update original dataframe with the new values
data.update(data_temp)

# get data where Vlo-I is different from Vlo-O
data_dif = data[data["Vlo-I"] != data["Vlo-O"]]
print(f'Cantidad de datos donde Vlo-I es diferente de Vlo-O: {data_dif.shape[0]}')
data_dif.sample(frac = 0.1, random_state = 42)

Cantidad de datos donde Vlo-I es diferente de Vlo-O: 102


C:\Users\Bastian\AppData\Local\Temp\ipykernel_8816\988833416.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_temp["Vlo-O"] = data_temp["Vlo-O"].map(lambda x: x[:-1] if x[-1] == '0' else x)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
11605,2017-02-20 21:40:00,4503,SCEL,SAEZ,LAN,2017-02-20 23:05:00,1003,SCEL,SAEZ,LAN,20.0,2.0,2017.0,Lunes,I,Grupo LATAM,Santiago,Buenos Aires
26712,2017-05-26 08:10:00,600,SCEL,SPJC,LAN,2017-05-26 08:17:00,116,SCEL,SPJC,LAN,26.0,5.0,2017.0,Viernes,I,Grupo LATAM,Santiago,Lima
17087,2017-03-26 14:00:00,204,SCEL,SPJC,LAW,2017-03-26 15:18:00,202,SCEL,SPJC,JMR,26.0,3.0,2017.0,Domingo,I,Latin American Wings,Santiago,Lima
16170,2017-03-15 23:20:00,532,SCEL,KJFK,LAN,2017-03-15 23:18:00,1318,SCEL,KJFK,LAN,15.0,3.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Nueva York
13907,2017-03-22 11:00:00,1071,SCEL,SCIE,SKU,2017-03-22 11:00:00,71,SCEL,SCIE,SKU,22.0,3.0,2017.0,Miercoles,N,Sky Airline,Santiago,Concepcion
11645,2017-02-17 01:15:00,702,SCEL,SPJC,LAW,2017-02-17 02:20:00,70,SCEL,SPJC,JMR,17.0,2.0,2017.0,Viernes,I,Latin American Wings,Santiago,Lima
39199,2017-08-17 17:25:00,624,SCEL,SPJC,AVA,2017-08-17 19:21:00,5924,SCEL,SPJC,TPU,17.0,8.0,2017.0,Jueves,I,Avianca,Santiago,Lima
16090,2017-03-18 23:35:00,500,SCEL,KMIA,LAN,2017-03-18 23:42:00,134,SCEL,KMIA,LAN,18.0,3.0,2017.0,Sabado,I,Grupo LATAM,Santiago,Miami
6095,2017-01-05 01:15:00,704,SCEL,SPJC,LAW,2017-01-05 01:15:00,708,SCEL,SPJC,56R,5.0,1.0,2017.0,Jueves,I,Latin American Wings,Santiago,Lima
6018,2017-01-05 07:05:00,4501,SCEL,SAEZ,LAN,2017-01-05 09:36:00,1001,SCEL,SAEZ,LAN,5.0,1.0,2017.0,Jueves,I,Grupo LATAM,Santiago,Buenos Aires


Vemos que los casos donde no coinciden los numeros de vuelo ya no se pueden arreglar facilmente, pero son muchos menos que antes, son solo 102

Pasando a verificar la integridad de otras columnas, revisamos 'DIA', 'MES', y 'AÑO'

In [160]:
# change dtype to int, and print unique values
data.loc[:, ['DIA', 'MES', 'AÑO']] = data[['DIA', 'MES', 'AÑO']].astype(int)
for col in ['DIA', 'MES', 'AÑO']:
    print(f'Column: {col}\n{data[col].unique()}')


Column: DIA
[ 1  2  3  4  5  7  8 10 12 14 16 18 21 23 25 28 30  6  9 11 13 15 17 19
 22 24 26 27 29 31 20]
Column: MES
[ 1  2  3  4  5  6  7  8  9 10 11 12]
Column: AÑO
[2017 2018]


Vemos que pueden ser convertidos en int sin problemas, y los numeros tienen sentido, dia va de 1 a 31, mes de 1 a 12, y año 2017 a 2018 

Revisamos ahora los valores unicos de las demas columnas con texto, por ser extenso, comenzamos por Ori, Des y Emp

In [161]:
for col in ['Ori-I', 'Ori-O', 'Des-I', 'Des-O', 'Emp-I', 'Emp-O']:
    print(f'Column: {col}\n{sorted(data[col].unique())}')

Column: Ori-I
['SCEL']
Column: Ori-O
['SCEL']
Column: Des-I
['CYYZ', 'EGLL', 'EGYP', 'KATL', 'KDFW', 'KIAH', 'KJFK', 'KLAX', 'KMCO', 'KMIA', 'LEMD', 'LFPG', 'LIRF', 'MDPC', 'MMMX', 'MMUN', 'MPTO', 'NZAA', 'SAAR', 'SABE', 'SACO', 'SAEZ', 'SAME', 'SANT', 'SANU', 'SARI', 'SAWH', 'SAZN', 'SAZS', 'SBCT', 'SBFI', 'SBFL', 'SBGL', 'SBGR', 'SCAR', 'SCAT', 'SCBA', 'SCCF', 'SCCI', 'SCDA', 'SCFA', 'SCIE', 'SCIP', 'SCJO', 'SCNT', 'SCPQ', 'SCQP', 'SCSE', 'SCTE', 'SCVD', 'SEGU', 'SEQM', 'SEQU', 'SGAS', 'SKBO', 'SLCB', 'SLLP', 'SLVR', 'SPJC', 'SPSO', 'SULS', 'SUMU', 'YMML', 'YSSY']
Column: Des-O
['CYYZ', 'EGLL', 'EGYP', 'KATL', 'KDFW', 'KIAD', 'KIAH', 'KJFK', 'KLAX', 'KMCO', 'KMIA', 'LEMD', 'LFPG', 'LIRF', 'MDPC', 'MMMX', 'MMUN', 'MPTO', 'NZAA', 'SAAR', 'SABE', 'SACO', 'SAEZ', 'SAME', 'SANT', 'SANU', 'SAWH', 'SAZN', 'SAZS', 'SBCT', 'SBFI', 'SBFL', 'SBGL', 'SBGR', 'SCAR', 'SCAT', 'SCBA', 'SCCF', 'SCCI', 'SCDA', 'SCFA', 'SCIE', 'SCIP', 'SCJO', 'SCNT', 'SCPQ', 'SCQP', 'SCSE', 'SCTE', 'SCVD', 'SEGU', 'SEQ

Vemos que en general las siglas son similares y tienen sentido, salvo por Emp-O que contiene '48O' y '56R' como sigla de aerolinea

In [162]:
data_emp_error = data[data['Emp-O'].isin(['48O', '56R'])]
print(f'Cantidad de datos donde Emp-O es 48O o 56R: {data_emp_error.shape[0]}')
data_emp_error.sample(frac = 0.2, random_state = 42)

Cantidad de datos donde Emp-O es 48O o 56R: 26


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6066,2017-01-16 11:00:00,200,SCEL,SPJC,LAW,2017-01-16 11:09:00,200,SCEL,SPJC,48O,16,1,2017,Lunes,I,Latin American Wings,Santiago,Lima
6086,2017-01-18 01:15:00,700,SCEL,SPJC,LAW,2017-01-18 01:36:00,702,SCEL,SPJC,56R,18,1,2017,Miercoles,I,Latin American Wings,Santiago,Lima
6058,2017-01-01 11:00:00,200,SCEL,SPJC,LAW,2017-01-01 11:07:00,200,SCEL,SPJC,48O,1,1,2017,Domingo,I,Latin American Wings,Santiago,Lima
6101,2017-01-19 01:15:00,704,SCEL,SPJC,LAW,2017-01-19 01:15:00,704,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima
6080,2017-01-06 01:15:00,700,SCEL,SPJC,LAW,2017-01-06 02:18:00,700,SCEL,SPJC,56R,6,1,2017,Viernes,I,Latin American Wings,Santiago,Lima


Efectivamente, estos parecen ser errores de algun tipo, de todas maneras, son solo 26 datos

Finalmente, solo queda comprobar que los tiempos de vuelo puedan ser convertidos a datetime

In [163]:
data['Fecha-I'] = pd.to_datetime(data['Fecha-I'], format='%Y-%m-%d %H:%M:%S')
data['Fecha-O'] = pd.to_datetime(data['Fecha-O'], format='%Y-%m-%d %H:%M:%S')
data.head(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


Y funciona sin problemas

# Guardar datos limpios

Ahora que le han solucionado todos los problemas encontrados, solo queda decidir que hacer con los casos donde el numero de vuelo programado y de operacion no coinciden, ademas de los casos donde la sigla de la empresa aerolinea contiene numeros (Lo cual podria no ser un error). Considerando que la cantidad de filas originales es 68206, y las lineas con problemas son 102 + 26, lo que corresponde a menos del 0.2% de los datos, estos simplemente se eliminarán.

In [166]:
# remove data where Vlo-I is different from Vlo-O
data = data[data["Vlo-I"] == data["Vlo-O"]]
# remove data where Emp-O is 48O or 56R
data = data[~data['Emp-O'].isin(['48O', '56R'])]
print(f'Shape of dataframe: {data.shape}')
data.head(5)

Shape of dataframe: (68081, 18)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


Guardamos los datos limpios entonces

In [167]:
data.to_csv(Path(os.getcwd()).joinpath('dataset_SCL_clean.csv'), index=False)